In [ ]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
import pandas as pd

from gen_planner.python.src import GenPlanner,func_zones, gen_plan

zones, roads = GenPlanner(gpd.read_file('1.gpkg'), rotation=0).terr2district2zone2block(gen_plan)


In [ ]:
zones.to_file('zones.gpkg')
roads.to_file('roads.gpkg')

In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [6]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from gen_planner.python.src import GenPlanner,basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).district2zone2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

<class 'bool'> True
POLYGON ((387358.046539353 6643287.263482422, 390508.3231588702 6642324.20928604, 391125.5481593057 6644343.233236258, 387975.27153978846 6645306.28743264, 387358.046539353 6643287.263482422))
-0.2966821620559408


In [5]:
from gen_planner.python.src import GenPlanner,basic_func_zone
import geopandas as gpd
zones, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).district2zone2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

<class 'bool'> True
POLYGON ((531227.1426911172 6733058.528637394, 528718.645433214 6729744.598117679, 530272.4791110614 6728568.415485929, 532780.9763689646 6731882.346005645, 531227.1426911172 6733058.528637394))
-2.218733984300628


In [2]:
from gen_planner.python.src import GenPlanner,basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).district2zone2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

<class 'bool'> True
POLYGON ((598704.8309976459 6772931.443388134, 603475.34757064 6775647.814811248, 601308.1600096836 6779453.849615015, 596537.6434366894 6776737.478191902, 598704.8309976459 6772931.443388134))
0.5176217661389989


In [1]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
from gen_planner.python.src import GenPlanner,basic_func_zone,terr_zones
import geopandas as gpd
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).zone2block(terr_zones.residential_terr)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

<class 'bool'> True
POLYGON ((598704.8309976459 6772931.443388134, 603475.34757064 6775647.814811248, 601308.1600096836 6779453.849615015, 596537.6434366894 6776737.478191902, 598704.8309976459 6772931.443388134))
0.5176217661389989


In [ ]:
0.3